In [ ]:
# ======================================================
# CARS 光伏 100 m 数据预处理 + 参数输入模板
# ======================================================

import os
import math
import pickle
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_origin
from tqdm import tqdm

# ======================================================
# 1. 路径设置
# ======================================================
base_dir = "CARS_PV_100m"
input_dir = os.path.join(base_dir, "input")
output_dir = os.path.join(base_dir, "output")
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 原始光伏与适宜性数据
pv_50m_file    = "PV/PV_50M_2022.tif"
expan_src_file = "PV_Suitability_Map/PV_Expan.tif"
loss_src_file  = "PV_Suitability_Map/PV_Loss.tif"

# 重采样/重投影输出路径（100 m 分辨率）
pv_100_file       = os.path.join(input_dir, "PV_100m_2022.tif")
expan_100_float   = os.path.join(input_dir, "PV_Expan_100m_float.tif")
loss_100_float    = os.path.join(input_dir, "PV_Loss_100m_float.tif")
expan_100_uint8   = os.path.join(input_dir, "PV_Expan_100m_uint8.tif")
loss_100_uint8    = os.path.join(input_dir, "PV_Loss_100m_uint8.tif")

# 参数保存文件
params_file       = os.path.join(base_dir, "params.pkl")

# 目标分辨率
target_res = 100.0  # meters

print("✅ 100 m 数据预处理路径已设置，PV 重采样采用单格占用原则（块内任意为 2，则整格为 2）。")


In [ ]:
# ======================================================
# 2. 基准网格（以 PV_50m bounds & CRS 为准，100 m 精度）
# ======================================================
with rasterio.open(pv_50m_file) as src:
    bounds = src.bounds
    crs = src.crs

minx, miny, maxx, maxy = bounds.left, bounds.bottom, bounds.right, bounds.top
target_width  = int(math.ceil((maxx - minx) / target_res))
target_height = int(math.ceil((maxy - miny) / target_res))
target_transform = from_origin(minx, maxy, target_res, target_res)

print(f"目标栅格: {target_width} x {target_height}, 分辨率 {target_res} m, CRS {crs}")


In [ ]:
# ======================================================
# 3. 分块重投影函数（带进度条）
# ======================================================
def reproject_tiled(src_path, dst_path, dst_dtype, resampling_method, dst_nodata=None, tile_height=256):
    """
    分块重投影栅格数据到目标网格，带进度条显示。
    dst_dtype: 输出数据类型
    resampling_method: 重采样方法（如 Resampling.mode, Resampling.bilinear）
    dst_nodata: 输出 nodata 值
    tile_height: 分块高度
    """
    with rasterio.open(src_path) as src:
        profile = {
            "driver": "GTiff",
            "height": target_height,
            "width": target_width,
            "count": 1,
            "dtype": dst_dtype,
            "crs": src.crs,
            "transform": target_transform,
            "compress": "lzw",
            "nodata": dst_nodata
        }

        # 初始化目标数组
        if np.issubdtype(np.dtype(dst_dtype), np.integer):
            fill = dst_nodata if dst_nodata is not None else 0
            dst_full = np.full((target_height, target_width), fill, dtype=dst_dtype)
        else:
            dst_full = np.full((target_height, target_width), np.nan, dtype=dst_dtype)

        n_tiles = int(math.ceil(target_height / tile_height))
        with tqdm(total=n_tiles, desc=f"Reproject {os.path.basename(src_path)}", unit="tile") as pbar:
            for t in range(n_tiles):
                y0 = t * tile_height
                y1 = min((t + 1) * tile_height, target_height)
                x0 = 0
                x1 = target_width

                win_transform = from_origin(minx + x0 * target_res,
                                            maxy - y0 * target_res,
                                            target_res, target_res)
                dst_window = dst_full[y0:y1, x0:x1]

                reproject(
                    source=rasterio.band(src, 1),
                    destination=dst_window,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=win_transform,
                    dst_crs=src.crs,
                    resampling=resampling_method,
                    num_threads=1
                )
                pbar.update(1)

        # 写出文件
        with rasterio.open(dst_path, 'w', **profile) as dst:
            dst.write(dst_full, 1)
    print(f"Saved -> {dst_path}")
    return dst_full


In [ ]:
# ======================================================
# 4. Step A: PV 分类重采样（100 m, 单格占用即算 PV）
# ======================================================
print("\nStep A: 重采样 PV（classification, mode）")

from rasterio.enums import Resampling

def classify_pv_postprocess(arr, nodata=0):
    """
    将数组中非 1、2 的值统一为 nodata
    """
    arr = arr.copy()
    arr[~np.isin(arr, [1, 2])] = nodata
    return arr

try:
    # mode 重采样，块内任一为 2 则算作 2
    pv_resampled = reproject_tiled(
        pv_50m_file,
        pv_100_file,
        dst_dtype='uint8',
        resampling_method=Resampling.mode,
        dst_nodata=0,
        tile_height=256
    )
except Exception as e:
    print(f"Mode resampling failed: {e}. 使用 nearest 替代")
    pv_resampled = reproject_tiled(
        pv_50m_file,
        pv_100_file,
        dst_dtype='uint8',
        resampling_method=Resampling.nearest,
        dst_nodata=0,
        tile_height=256
    )

# 后处理，确保只有 1/2/0（nodata）
pv_resampled = classify_pv_postprocess(pv_resampled, nodata=0)


In [ ]:
# ======================================================
# 5. Step B: 适宜性双线性重采样（100 m -> float32）
# ======================================================
print("\nStep B: 重采样适宜性（bilinear -> float32）")

expan_float_arr = reproject_tiled(
    expan_src_file,
    expan_100_float,
    dst_dtype='float32',
    resampling_method=Resampling.bilinear,
    dst_nodata=np.nan,
    tile_height=128
)

loss_float_arr = reproject_tiled(
    loss_src_file,
    loss_100_float,
    dst_dtype='float32',
    resampling_method=Resampling.bilinear,
    dst_nodata=np.nan,
    tile_height=128
)


In [ ]:
# ======================================================
# 6. Step C: min-max 归一化到 0-255 uint8（nodata=255）
# ======================================================
def normalize_to_uint8_strict(in_array, out_path, transform, crs,
                              valid_min=0.0, valid_max=100.0, nodata_val=255):
    valid_mask = np.isfinite(in_array) & (in_array >= valid_min) & (in_array <= valid_max)
    out = np.full(in_array.shape, nodata_val, dtype=np.uint8)

    if np.any(valid_mask):
        vmin = np.nanmin(in_array[valid_mask])
        vmax = np.nanmax(in_array[valid_mask])
        print(f"{os.path.basename(out_path)} 有效范围: {vmin:.4f} – {vmax:.4f}")

        if vmax == vmin:
            out[valid_mask] = 127
        else:
            for i in tqdm(range(in_array.shape[0]), desc=f"Normalizing {os.path.basename(out_path)}"):
                row_valid = valid_mask[i, :]
                out[i, row_valid] = np.round((in_array[i, row_valid] - vmin) / (vmax - vmin) * 255).astype(np.uint8)

    profile = {
        "driver": "GTiff",
        "height": out.shape[0],
        "width": out.shape[1],
        "count": 1,
        "dtype": "uint8",
        "crs": crs,
        "transform": transform,
        "compress": "lzw",
        "nodata": nodata_val
    }
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(out, 1)
    print(f"Saved: {out_path}")
    return out

# 将 150 m 分辨率的适宜性浮点栅格归一化
expan_150_uint8_arr = normalize_to_uint8_strict(expan_float_arr, expan_100_uint8, target_transform, crs)
loss_150_uint8_arr  = normalize_to_uint8_strict(loss_float_arr, loss_100_uint8, target_transform, crs)


In [ ]:
# ======================================================
# 7. Step D: 对齐检查
# ======================================================
print("\nStep D: 对齐检查")
with rasterio.open(pv_100_file) as a, \
     rasterio.open(expan_100_uint8) as b, \
     rasterio.open(loss_100_uint8) as c:
    print("PV_150:", (a.width, a.height), "transform:", a.transform)
    print("Expan_150_uint8:", (b.width, b.height), "transform:", b.transform)
    print("Loss_150_uint8:", (c.width, c.height), "transform:", c.transform)

print("\n重采样完成，输出目录：", input_dir)


In [ ]:
import os
import math
import rasterio

# ======================================================
# ---------- 输入输出路径 ----------
# ======================================================
base_dir = "CARS_PV_100m"
input_dir = os.path.join(base_dir, "input")
os.makedirs(input_dir, exist_ok=True)

pv_100m_file = os.path.join(input_dir, "PV_100m_2022.tif")
params_out = os.path.join(input_dir, "cars_params.txt")

# ---------- 每像元面积（km²） ----------
pixel_area_km2 = 0.01  # 100mx100m -> 0.01 km²

# ======================================================
# ---------- 计算当前光伏面积 ----------
# ======================================================
with rasterio.open(pv_100m_file) as src:
    pv_data = src.read(1)
    nodata = src.nodata if src.nodata is not None else 0
    # 光伏像元标记为 2
    pv_mask = (pv_data == 2)
    current_area_km2 = pv_mask.sum() * pixel_area_km2

print(f"\n当前光伏面积计算结果: {current_area_km2:.4f} km² ({pv_mask.sum():,} 像元)")

# ======================================================
# ---------- 参数说明 ----------
# ======================================================
print("\n请设置 CARS 模拟参数（按 Enter 使用默认值）。")
print("\n参数说明:")
print(" - mode: 模拟模式 (expansion=扩张, contraction=收缩, auto=自动选择)")
print(" - target_area: 目标光伏面积 (km²)，越大表示越多光伏布局")
print(" - alpha: 适宜性权重 α (0-1), 越大越倾向于高适宜区")
print(" - beta: 邻域权重 β (0-1), 越大越考虑邻近现有光伏")
print(" - gamma: 随机权重 γ (0-1), 越大随机性越高")
print(" - neigh_size: 邻域窗口大小 (奇数, 3,5,7...), 越大局部扩张连通性越强")
print(" - convert_frac: 每次迭代转换比例 (0-1), 越大迭代转换像元越多")
print(" - max_iters: 最大迭代次数，越大允许搜索更多迭代")
print(" - selection: 选择策略 (topk=取概率最高, prob=按概率抽样)")

# ======================================================
# ---------- 输入函数 ----------
# ======================================================
def ask(prompt, default, t=float):
    s = input(f"{prompt} (默认 {default}): ").strip()
    if s == "":
        return default
    try:
        return t(s)
    except:
        print("输入解析失败，使用默认值。")
        return default

# ======================================================
# ---------- 用户输入 ----------
# ======================================================
mode = ask("模拟模式", "auto", str).lower()
target_area = ask(f"目标光伏面积 (km²)，>= 当前 {current_area_km2:.3f}", current_area_km2, float)
alpha = ask("alpha (适宜性权重)", 0.6, float)
beta = ask("beta (邻域权重)", 0.3, float)
gamma = ask("gamma (随机权重)", 0.1, float)
neigh_size = ask("邻域窗口大小", 3, int)
convert_frac = ask("每次迭代转换比例 convert_frac", 0.1, float)
max_iters = ask("最大迭代次数 max_iters", 500, int)
selection = ask("选择策略 selection", "topk", str)

# ======================================================
# ---------- 权重归一化 ----------
# ======================================================
s = alpha + beta + gamma
if s <= 0:
    alpha, beta, gamma = 0.6, 0.3, 0.1
else:
    alpha, beta, gamma = alpha/s, beta/s, gamma/s

# ======================================================
# ---------- 自动模式判断 ----------
# ======================================================
if mode == "auto":
    if target_area > current_area_km2:
        mode = "expansion"
    elif target_area < current_area_km2:
        mode = "contraction"
    else:
        mode = "none"

# ======================================================
# ---------- 估算需要转换像元数 ----------
# ======================================================
if target_area >= current_area_km2:
    need_cells = int(math.ceil((target_area - current_area_km2) / pixel_area_km2))
else:
    need_cells = int(math.ceil((current_area_km2 - target_area) / pixel_area_km2))

# ======================================================
# ---------- 打印确认 ----------
# ======================================================
print("\n=== 参数确认 ===")
print(f"模式: {mode}")
print(f"当前光伏面积: {current_area_km2:.4f} km²")
print(f"目标面积: {target_area:.4f} km²")
print(f"需转换像元数(估算): {need_cells:,}")
print(f"alpha={alpha:.3f}, beta={beta:.3f}, gamma={gamma:.3f}, neigh_size={neigh_size}, convert_frac={convert_frac}, max_iters={max_iters}, selection={selection}")

# ======================================================
# ---------- 保存参数到文件 ----------
# ======================================================
os.makedirs(os.path.dirname(params_out), exist_ok=True)
with open(params_out, "w", encoding="utf-8") as f:
    f.write("param\tvalue\n")
    f.write(f"pv_clip_path\t{pv_100m_file}\n")
    f.write(f"current_area_km2\t{current_area_km2}\n")
    f.write(f"mode\t{mode}\n")
    f.write(f"target_area_km2\t{target_area}\n")
    f.write(f"need_cells_est\t{need_cells}\n")
    f.write(f"alpha\t{alpha}\n")
    f.write(f"beta\t{beta}\n")
    f.write(f"gamma\t{gamma}\n")
    f.write(f"neigh_size\t{neigh_size}\n")
    f.write(f"convert_frac\t{convert_frac}\n")
    f.write(f"max_iters\t{max_iters}\n")
    f.write(f"selection\t{selection}\n")

print("\n参数已保存 ->", params_out)

# ======================================================
# ---------- 保存到字典以便后续调用 ----------
# ======================================================
cars_params = {
    "pv_clip_path": pv_100m_file,
    "current_area_km2": current_area_km2,
    "mode": mode,
    "target_area_km2": target_area,
    "need_cells_est": need_cells,
    "alpha": alpha,
    "beta": beta,
    "gamma": gamma,
    "neigh_size": neigh_size,
    "convert_frac": convert_frac,
    "max_iters": max_iters,
    "selection": selection
}

print("\n参数字典 cars_params 已生成，可在后续模拟中直接使用。")

# ======================================================
# ---------- 提示：可分前中后期模拟 ----------
# ======================================================
print("\n提示：目标光伏面积可在后续模拟中按前/中/后期分步设置。")


In [ ]:
import os
import math
import numpy as np
import rasterio
from scipy.signal import convolve2d
from tqdm import tqdm

base_dir = "CARS_PV_100m"
input_dir = os.path.join(base_dir, "input")
output_dir = os.path.join(base_dir, "output")
os.makedirs(output_dir, exist_ok=True)
tmp_dir = os.path.join(output_dir, "tmp")
os.makedirs(tmp_dir, exist_ok=True)

pv_file = os.path.join(input_dir, "PV_100m_2022.tif")
expan_file = os.path.join(input_dir, "PV_Expan_100m_uint8.tif")
loss_file = os.path.join(input_dir, "PV_Loss_100m_uint8.tif")
pv_out_file = os.path.join(output_dir, "PV_CA_100m_future.tif")

mode = cars_params["mode"]
target_area_km2 = cars_params["target_area_km2"]
alpha = cars_params["alpha"]
beta = cars_params["beta"]
gamma = cars_params["gamma"]
neigh_size = cars_params["neigh_size"]
convert_frac = cars_params["convert_frac"]
max_iters = cars_params["max_iters"]
selection = cars_params["selection"]

# 读取栅格
with rasterio.open(pv_file) as src:
    pv_sim = src.read(1)
    profile = src.profile
    pixel_area_km2 = abs(src.transform[0]*src.transform[4]) / 1e6

with rasterio.open(expan_file) as src:
    expan = src.read(1)
with rasterio.open(loss_file) as src:
    loss = src.read(1)

current_cells = np.sum(pv_sim==2)
current_area_km2 = current_cells * pixel_area_km2
print(f"当前光伏面积: {current_area_km2:.3f} km² ({current_cells:,} 像元)")

if mode=="expansion":
    total_need_cells = int(math.ceil((target_area_km2 - current_area_km2)/pixel_area_km2))
    S_map = expan
else:
    total_need_cells = int(math.ceil((current_area_km2 - target_area_km2)/pixel_area_km2))
    S_map = loss

kernel = np.ones((neigh_size, neigh_size), dtype=np.float32)
kernel /= kernel.sum()

rows, cols = pv_sim.shape
converted_cells = 0
iter_count = 0

print("\n开始 CARS 模拟...")

with tqdm(total=total_need_cells, desc="转换进度", unit="cell") as pbar:
    while converted_cells < total_need_cells and iter_count < max_iters:
        iter_count += 1

        # 邻域权重
        neighbor_mask = (pv_sim==2).astype(np.float32)
        N_full = convolve2d(neighbor_mask, kernel, mode='same', boundary='fill', fillvalue=0)

        # 随机抽取一个块，避免存储全量 candidates
        block_rows = 1000  # 每次只取1000行
        r_start = np.random.randint(0, rows-block_rows)
        r_end = r_start + block_rows
        c_start = 0
        c_end = cols

        block = pv_sim[r_start:r_end, c_start:c_end]
        N_block = N_full[r_start:r_end, c_start:c_end]
        S_block = S_map[r_start:r_end, c_start:c_end]/255.0

        if mode=="expansion":
            mask = (block != 2)
        else:
            mask = (block == 2)

        candidate_indices = np.argwhere(mask)
        if len(candidate_indices)==0:
            continue

        N_vals = N_block[mask]
        S_vals = S_block[mask]
        R_vals = np.random.rand(len(candidate_indices))
        P_vals = alpha*S_vals + beta*N_vals + gamma*R_vals

        # 按概率选择
        remain_cells = total_need_cells - converted_cells
        num_convert = max(1, int(round(convert_frac*remain_cells)))
        num_convert = min(num_convert, len(P_vals))

        if selection=="topk":
            idx_selected = np.argpartition(P_vals, -num_convert)[-num_convert:]
        else:
            P_sum = P_vals.sum()
            if P_sum==0:
                continue
            idx_selected = np.random.choice(len(P_vals), size=num_convert, replace=False, p=P_vals/P_sum)

        for idx in idx_selected:
            r, c = candidate_indices[idx]
            global_r = r_start + r
            global_c = c_start + c
            if mode=="expansion" and pv_sim[global_r, global_c] !=2:
                pv_sim[global_r, global_c] = 2
                converted_cells += 1
                pbar.update(1)
            elif mode=="contraction" and pv_sim[global_r, global_c]==2:
                pv_sim[global_r, global_c] = 0
                converted_cells += 1
                pbar.update(1)

        # 每5次迭代保存中间结果
        if iter_count % 5==0:
            tmp_file = os.path.join(tmp_dir, f"PV_CA_iter{iter_count}_100m.tif")
            profile.update(dtype=rasterio.uint8, count=1, compress="lzw", nodata=0)
            with rasterio.open(tmp_file, 'w', **profile) as dst:
                dst.write(pv_sim.astype(np.uint8), 1)

sim_area = np.sum(pv_sim==2)*pixel_area_km2
print(f"\n最终光伏面积: {sim_area:.3f} km² ({np.sum(pv_sim==2):,} 像元)")

# 保存最终结果
profile.update(dtype=rasterio.uint8, count=1, compress="lzw", nodata=0)
with rasterio.open(pv_out_file, 'w', **profile) as dst:
    dst.write(pv_sim.astype(np.uint8), 1)
print(f"最终模拟结果已保存 -> {pv_out_file}")
